In [1]:
!pwd

/Users/shihosato/src/github.com/twinkle13531/master_degree/202012/QA


In [2]:
%load_ext autoreload
%autoreload 2

In [68]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import statistics
import math
import datetime
%matplotlib inline

## Amazon Braket

In [4]:
from braket.ocean_plugin.braket_dwave_sampler import BraketDWaveSampler
s3_destination_folder = ('amazon-braket-ecc806acea4c', 'qaExactLogisticRegression_ssato')
dw = BraketDWaveSampler(s3_destination_folder, device_arn='arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')

In [5]:
from dwave.system import AutoEmbeddingComposite

In [6]:
# 実効例です。節約のためコメントアウトしています。
from dimod import BQM
Q = np.array([
      [ 1,  2, -1,  0],
      [ 0, -1, -2, -1],
      [ 0,  0, -2,  2],
      [ 0,  0,  0,  0]])
bqm = BQM.from_qubo(Q)

In [7]:
bqm

BinaryQuadraticModel({0: 1.0, 1: -1.0, 2: -2.0, 3: 0.0}, {(0, 1): 2, (0, 2): -1, (1, 2): -2, (1, 3): -1, (2, 3): 2}, 0, 'BINARY')

In [8]:
dw_sampler = AutoEmbeddingComposite(dw)

res = dw_sampler.sample(bqm, num_reads=1, annealing_time=3)

In [9]:
res

SampleSet(rec.array([([0, 1, 1, 0], -5., 1, 0.)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i8'), ('chain_break_fraction', '<f8')]), [0, 1, 2, 3], {'taskMetadata': {'braketSchemaHeader': {'name': 'braket.task_result.task_metadata', 'version': '1'}, 'id': 'arn:aws:braket:us-west-2:160217898313:quantum-task/3f7eef47-ecfe-4f0f-bd14-58e839648699', 'shots': 1, 'deviceId': 'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6', 'deviceParameters': {'braketSchemaHeader': {'name': 'braket.device_schema.dwave.dwave_device_parameters', 'version': '1'}, 'providerLevelParameters': {'braketSchemaHeader': {'name': 'braket.device_schema.dwave.dwave_provider_level_parameters', 'version': '1'}, 'annealingOffsets': None, 'annealingSchedule': None, 'annealingDuration': 3, 'autoScale': None, 'beta': None, 'chains': None, 'compensateFluxDrift': None, 'fluxBiases': None, 'initialState': None, 'maxResults': None, 'postprocessingType': None, 'programmingThermalizationDuration'

## aws情報をpyファイルに書いたので使ってみる（aws_QA_DwaveSampler_AutoEmbedding.py）

In [11]:
import aws_QA_DwaveSampler_AutoEmbedding as awsqa

In [15]:
chain_strength_dic = {20:10, 25:15, 30:20, 35:20, 40:30}

In [18]:
bit, num = 20, 3
num_reads = 100
chain_strength = chain_strength_dic[bit]
path = '../input/ost{}_{}.csv'.format(bit, num)
df = pd.read_csv(path, sep=',')
bqm = awsqa.make_Hamiltonian(df)
res = awsqa.sample_(bqm, chain_strength, num_reads)

In [19]:
res

SampleSet(rec.array([([0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1],  0., 1, 0.  ),
           ([0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0],  1., 1, 0.  ),
           ([1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1],  1., 1, 0.  ),
           ([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0],  1., 1, 0.  ),
           ([1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1],  1., 1, 0.  ),
           ([1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1],  1., 1, 0.  ),
           ([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1],  1., 1, 0.  ),
           ([0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1],  1., 1, 0.  ),
           ([1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],  1., 1, 0.  ),
           ([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1],  1., 1, 0.  ),
           ([0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1],  1., 1, 0.  ),
           

In [20]:
res.record

rec.array([([0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1],  0., 1, 0.  ),
           ([0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0],  1., 1, 0.  ),
           ([1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1],  1., 1, 0.  ),
           ([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0],  1., 1, 0.  ),
           ([1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1],  1., 1, 0.  ),
           ([1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1],  1., 1, 0.  ),
           ([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1],  1., 1, 0.  ),
           ([0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1],  1., 1, 0.  ),
           ([1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],  1., 1, 0.  ),
           ([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1],  1., 1, 0.  ),
           ([0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1],  1., 1, 0.  ),
           ([1, 1, 1,

In [21]:
res.info

{'taskMetadata': {'braketSchemaHeader': {'name': 'braket.task_result.task_metadata',
   'version': '1'},
  'id': 'arn:aws:braket:us-west-2:160217898313:quantum-task/74bbf256-dc19-4392-af4d-14d38e0b8bd7',
  'shots': 100,
  'deviceId': 'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6',
  'deviceParameters': {'braketSchemaHeader': {'name': 'braket.device_schema.dwave.dwave_device_parameters',
    'version': '1'},
   'providerLevelParameters': {'braketSchemaHeader': {'name': 'braket.device_schema.dwave.dwave_provider_level_parameters',
     'version': '1'},
    'annealingOffsets': None,
    'annealingSchedule': None,
    'annealingDuration': None,
    'autoScale': None,
    'beta': None,
    'chains': None,
    'compensateFluxDrift': None,
    'fluxBiases': None,
    'initialState': None,
    'maxResults': None,
    'postprocessingType': None,
    'programmingThermalizationDuration': None,
    'readoutThermalizationDuration': None,
    'reduceIntersampleCorrelation': None,
    'reinitializeS

In [24]:
res.info.keys()

dict_keys(['taskMetadata', 'additionalMetadata'])

In [25]:
res.info['taskMetadata']

{'braketSchemaHeader': {'name': 'braket.task_result.task_metadata',
  'version': '1'},
 'id': 'arn:aws:braket:us-west-2:160217898313:quantum-task/74bbf256-dc19-4392-af4d-14d38e0b8bd7',
 'shots': 100,
 'deviceId': 'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6',
 'deviceParameters': {'braketSchemaHeader': {'name': 'braket.device_schema.dwave.dwave_device_parameters',
   'version': '1'},
  'providerLevelParameters': {'braketSchemaHeader': {'name': 'braket.device_schema.dwave.dwave_provider_level_parameters',
    'version': '1'},
   'annealingOffsets': None,
   'annealingSchedule': None,
   'annealingDuration': None,
   'autoScale': None,
   'beta': None,
   'chains': None,
   'compensateFluxDrift': None,
   'fluxBiases': None,
   'initialState': None,
   'maxResults': None,
   'postprocessingType': None,
   'programmingThermalizationDuration': None,
   'readoutThermalizationDuration': None,
   'reduceIntersampleCorrelation': None,
   'reinitializeState': None,
   'resultFormat': None,
  

In [26]:
res.info['additionalMetadata']

{'action': {'braketSchemaHeader': {'name': 'braket.ir.annealing.problem',
   'version': '1'},
  'type': <ProblemType.QUBO: 'QUBO'>,
  'linear': {474: 33.33333333333333,
   358: 32.33333333333333,
   346: 36.33333333333333,
   602: 11.333333333333332,
   350: 37.33333333333333,
   366: 10.333333333333332,
   629: 35.83333333333333,
   621: 32.83333333333333,
   613: 32.83333333333333,
   605: 33.83333333333333,
   597: 14.833333333333332,
   625: 17.833333333333332,
   623: 13.142857142857142,
   599: 38.14285714285714,
   593: 36.14285714285714,
   607: 35.14285714285714,
   465: 36.14285714285714,
   337: 19.142857142857142,
   615: 34.14285714285714,
   612: 34.16666666666667,
   601: 36.16666666666667,
   604: 56.16666666666667,
   473: 16.166666666666668,
   596: 16.166666666666668,
   620: 13.166666666666668,
   618: 31.33333333333333,
   490: 30.33333333333333,
   362: 12.333333333333332,
   874: 36.33333333333333,
   879: 16.833333333333332,
   746: 33.83333333333333,
   478: 35

In [27]:
res.info['additionalMetadata'].keys()

dict_keys(['action', 'dwaveMetadata', 'rigettiMetadata', 'simulatorMetadata'])

In [29]:
res.info['additionalMetadata']['dwaveMetadata'].keys()

dict_keys(['braketSchemaHeader', 'activeVariables', 'timing'])

In [30]:
res.info['additionalMetadata']['dwaveMetadata']['timing']

{'qpuSamplingTime': 23894,
 'qpuAnnealTimePerSample': 20,
 'qpuAccessTime': 34846,
 'qpuAccessOverheadTime': 2638,
 'qpuReadoutTimePerSample': 198,
 'qpuProgrammingTime': 10952,
 'qpuDelayTimePerSample': 21,
 'postProcessingOverheadTime': 1114,
 'totalPostProcessingTime': 1114,
 'totalRealTime': 34846,
 'runTimeChip': 23894,
 'annealTimePerRun': 20,
 'readoutTimePerRun': 198}

In [41]:
res.info['additionalMetadata']['dwaveMetadata']['timing'].keys()

dict_keys(['qpuSamplingTime', 'qpuAnnealTimePerSample', 'qpuAccessTime', 'qpuAccessOverheadTime', 'qpuReadoutTimePerSample', 'qpuProgrammingTime', 'qpuDelayTimePerSample', 'postProcessingOverheadTime', 'totalPostProcessingTime', 'totalRealTime', 'runTimeChip', 'annealTimePerRun', 'readoutTimePerRun'])

In [43]:
res.info['additionalMetadata']['dwaveMetadata']['timing']['qpuAccessTime']

34846

In [44]:
res.info['additionalMetadata']['dwaveMetadata']['timing']['qpuAccessTime']*10**(-6)

0.034845999999999995

TypeError: 'method' object is not subscriptable

In [40]:
for sample, energy, num_occurrences, chain_break_fraction in res.data(['sample', 'energy', 'num_occurrences', 'chain_break_fraction']):
    #print(list(sample.values()))
    #print(energy)
    #print(num_occurrences)
    #print(chain_break_fraction)

SyntaxError: unexpected EOF while parsing (<ipython-input-40-0d64c1e98d34>, line 5)

## num_summaryのための事前準備

In [69]:
print(datetime.datetime.now())

2021-01-13 21:16:35.069868


In [46]:
import aws_QA_DwaveSampler_AutoEmbedding as aqa

In [47]:
#chain_strength_list = [5, 10, 15, 20, 25, 30, 35]
chain_strength_dic = {20:10, 25:15, 30:20, 35:20, 40:30}

In [48]:
bit_list = [20, 25, 30, 35, 40]
num_list = [0, 1, 2, 3, 4]
n, num_reads = 1, 10000

In [49]:
bit = 20
num = 1

In [50]:
path = '../input/ost{}_{}.csv'.format(bit, num)
df = pd.read_csv(path, sep=',')
bqm = aqa.make_Hamiltonian(df)
res = aqa.aws_get_response(bqm, chain_strength, num_reads)
ads = aqa.aws_DSampler(df, bqm, n, num_reads, chain_strength, res)
valid_y_info_dic = ads.find_valid_y()

In [51]:
valid_y_info_dic

{(0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1): [1, 0.0],
 (1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0): [1, 0.0],
 (1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0): [5, 0.0],
 (1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0): [2, 0.0],
 (0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0): [1, 0.0],
 (0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0): [1, 0.0],
 (0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0): [1, 0.0],
 (0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0): [1, 0.0],
 (0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1): [1, 0.0],
 (1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0): [1, 0.0],
 (0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0): [1, 0.0],
 (1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0): [1, 0.0],
 (0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1): [1, 0.0],
 (0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0

In [52]:
occurrence_list = [valid_y_info[0] for valid_y_info in list(valid_y_info_dic.values())]

In [53]:
occurrence_list

[1,
 1,
 5,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [54]:
broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]

In [55]:
broken_list

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.05,
 0.05,
 0.05,
 0.05,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [56]:
time_dic = {}
y_dic = {}
y_dic['total_y_num'] = {}
y_dic['mean_occurrence'] = {}
y_dic['valid_y_num'] = {}
per_time_dic = {}
#mean_broken_chain_proportion = {}
p_dic = {}

In [57]:
time_dic['QA'] = {}
y_dic['total_y_num']['QA'] = {}
y_dic['mean_occurrence']['QA'] = {}
y_dic['valid_y_num']['QA'] = {}
per_time_dic['QA'] = {}
mean_broken_chain_proportion = {}
p_dic['QA'] = {}

In [64]:
num_reads=10000

In [65]:
#n=1でのみ通用するコード
for bit in [20, 30]:
    time_dic['QA'][bit] = {}
    y_dic['total_y_num']['QA'][bit] = {}
    y_dic['mean_occurrence']['QA'][bit] = {}
    y_dic['valid_y_num']['QA'][bit] = {}
    per_time_dic['QA'][bit] = {}
    mean_broken_chain_proportion[bit] = {}
    p_dic['QA'][bit] = {}
    chain_strength = chain_strength_dic[bit]
    
    for num in [1, 3]:
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        bqm = aqa.make_Hamiltonian(df)
        res = aqa.aws_get_response(bqm, chain_strength, num_reads)
        ads = aqa.aws_DSampler(df, bqm, num_reads, chain_strength, res)
        valid_y_info_dic = ads.find_valid_y()
        
        if len(valid_y_info_dic) > 0:
            calculation_time = res.info['additionalMetadata']['dwaveMetadata']['timing']['qpuAccessTime']*10**(-6)
            occurrence_list = [valid_y_info[0] for valid_y_info in list(valid_y_info_dic.values())]
            broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]

            time_dic['QA'][bit][num] = calculation_time
            y_dic['total_y_num']['QA'][bit][num] = sum(occurrence_list)
            y_dic['mean_occurrence']['QA'][bit][num] = statistics.mean(occurrence_list)
            y_dic['valid_y_num']['QA'][bit][num] = len(valid_y_info_dic)
            #per_time_dic['QA'][bit][num] = calculation_time / sum(occurrence_list)
            per_time_dic['QA'][bit][num] = calculation_time / len(valid_y_info_dic)
            mean_broken_chain_proportion[bit][num] = statistics.mean(broken_list)
            p_dic['QA'][bit][num] = ads.p_value(valid_y_info_dic)

    print('{} finish'.format(bit))

20 finish
30 finish


In [ ]:
print(datetime.datetime.now())

In [ ]:
per_time_dic['QA']

In [71]:
print(datetime.datetime.now())

2021-01-13 21:18:33.278015


In [66]:
per_time_dic['QA']

{20: {1: 0.028575297619047618, 3: 0.013045336956521738},
 30: {1: 0.600119, 3: 0.2667157777777778}}

In [67]:
time_dic['QA']

{20: {1: 2.400325, 3: 2.4003419999999998}, 30: {1: 2.400476, 3: 2.400442}}

In [63]:
y_dic['mean_occurrence']['QA']

{20: {1: 1, 3: 1}, 30: {}}

In [ ]:
broken_chain_proportion_dic = {}
valid_y_num_dic = {}
for bit in [20 ,30]:
    broken_chain_proportion_dic[bit] = {}
    valid_y_num_list[bit] = {}
    for chain_strength in [3, 10, 30]:
        broken_chain_proportion_list[bit][chain_strength] = []
        valid_y_num_list[bit][chain_strength] = []
    for num in [2, 3]:
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        bqm = qda.make_Hamiltonian(df)
        
        for chain_strength in [3, 10, 30]:
            qqda = qda.QA_DSampler_AEmbedding(df, bqm, n, num_reads, chain_strength)
            valid_y_info_dic, calculation_time = qqda.find_valid_y_time_ntimes()

            if len(valid_y_info_dic) > 0:
                broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]
                broken_chain_proportion_list[bit][chain_strength].append(statistics.mean(broken_list))
                valid_y_num_list[bit][chain_strength].append(len(valid_y_info_dic))
        
        print('{}_{} finished'.format(bit, num))

In [ ]:
path = '../input/ost{}_{}.csv'.format(bit, num)
df = pd.read_csv(path, sep=',')
bqm = qda.make_Hamiltonian(df)

In [8]:
broken_chain_proportion_list = {}
valid_y_num_list = {}
for bit in [20 ,30]:
    broken_chain_proportion_list[bit] = {}
    valid_y_num_list[bit] = {}
    for chain_strength in [3, 10, 30]:
        broken_chain_proportion_list[bit][chain_strength] = []
        valid_y_num_list[bit][chain_strength] = []
    for num in [2, 3]:
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        bqm = qda.make_Hamiltonian(df)
        
        for chain_strength in [3, 10, 30]:
            qqda = qda.QA_DSampler_AEmbedding(df, bqm, n, num_reads, chain_strength)
            valid_y_info_dic, calculation_time = qqda.find_valid_y_time_ntimes()

            if len(valid_y_info_dic) > 0:
                broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]
                broken_chain_proportion_list[bit][chain_strength].append(statistics.mean(broken_list))
                valid_y_num_list[bit][chain_strength].append(len(valid_y_info_dic))
        
        print('{}_{} finished'.format(bit, num))

20_2 finished
20_3 finished
30_2 finished
30_3 finished
